In [2]:
import pandas as pd
import glob
import re
import os
import requests
import time
from tqdm import tqdm


df_s = pd.read_csv("article_data.csv", sep=",", header=0)
#parse out beginning page
df_s[['BP', 'EP']] = df_s['pages'].str.split('-', expand=True)
#df_s = pd.read_csv(soc10_d, sep=",", header4=0, usecols = ["journal","volume", "issue", "pages"])
df_s



,artid,journal,volume,issue,year,month,pages,cited3,cited2012,aut_num,...,bip_avgdegree,aut_flowhier,ack_flowhier,com_flowhier,aut_numcomp,ack_numcomp,com_numcomp,bip_numcomp,BP,EP
0,1,annurevisoci,35,1,2009,6,1-19,0,0,1,...,0.000000,99.0,99.0,99.000000,1,0,1,1,1,19
1,2,annurevisoci,35,1,2009,6,21-42,15,15,1,...,0.000000,99.0,99.0,99.000000,1,0,1,1,21,42
2,3,annurevisoci,35,1,2009,6,43-65,17,21,2,...,0.777778,1.0,0.2,0.277778,1,3,3,5,43,65
3,4,annurevisoci,35,1,2009,6,67-85,6,7,1,...,0.666667,99.0,99.0,0.000000,1,2,2,2,67,85
4,5,annurevisoci,35,1,2009,6,87-105,18,18,2,...,0.000000,0.0,99.0,0.000000,1,0,1,2,87,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3593,3746,amersocirevi,65,1,2000,2,122-127,3,21,3,...,4.250000,0.0,99.0,0.000000,1,1,1,1,122,127
3594,3747,amersocirevi,65,1,2000,2,138-167,8,54,1,...,0.333333,99.0,99.0,0.000000,1,5,5,5,138,167
3595,3748,amersocirevi,65,1,2000,2,128-137,1,7,4,...,0.000000,1.0,99.0,1.000000,3,1,4,5,128,137
3596,3749,amersocirevi,65,1,2000,2,96-116,16,186,3,...,4.250000,0.0,99.0,0.000000,1,1,1,1,96,116


In [3]:

wos_meta_d = "WoS_sociology_metadata_2000_2009/"
dfs = []
for r, d, f0 in os.walk(wos_meta_d):
    for fname in f0:
        if "txt" in fname:
            df0 = pd.read_csv( os.path.join(wos_meta_d,fname), sep='\t', header=0, dtype={"BP":str, "EP":str} )
            dfs.append(df0)
df = pd.concat(dfs)
df = df.reset_index(drop=True)
df


,PT,AU,BA,CA,GP,RI,OI,BE,Z2,AU.1,...,DG,C1,C3,EC,DE,DA,UT,PM,Unnamed: 78,Publication Type
0,J,"Hallinan, Maureen T.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-07-01,WOS:000258547900003,NaN,NaN,NaN
1,J,"Jasper, James M.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-07-01,WOS:000258833100019,NaN,NaN,NaN
2,J,"Jean, Peter K. B. St.; Weitzer, Ronald",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-07-01,WOS:000258833100017,NaN,NaN,NaN
3,J,"Kaminski, Elizabeth",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-07-01,WOS:000258833100020,NaN,NaN,NaN
4,J,"Kempner, Joanna",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-07-01,WOS:000258833100010,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dissertation and Thesis Subjects
6312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Author Keywords
6313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Indexed Date
6314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UT (Unique ID)


In [4]:

for idx, c in enumerate(df.columns):
    print(idx+1, c)


1 PT
2 AU
3 BA
4 CA
5 GP
6 RI
7 OI
8 BE
9 Z2
10 AU.1
11 AA
12 TI
13 X1
14 Y1
15 Z1
16 FT
17 PN
18 AE
19 Z3
20 SO
21 S1
22 SE
23 BS
24 VL
25 IS
26 SI
27 MA
28 BP
29 EP
30 AR
31 VN
32 VH
33 DI
34 D2
35 L1
36 L2
37 L3
38 EA
39 SU
40 DT
41 PD
42 PY
43 AB
44 X4
45 Y4
46 Z4
47 AK
48 CT
49 CY
50 SP
51 CL
52 TC
53 Z8
54 ZR
55 ZA
56 ZB
57 ZS
58 Z9
59 U1
60 U2
61 SN
62 EI
63 BN
64 G1
65 NR
66 CR
67 LA
68 AS
69 AC
70 CG
71 DG
72 C1
73 C3
74 EC
75 DE
76 DA
77 UT
78 PM
79 Unnamed: 78
80 Publication Type


In [5]:

df1 = df[["DI", "D2", "AU","TI","SO","PY","VL","IS","BP","EP"]] # DOI, Authors, Title, Source, Publication Year, Volume, Issue, Begin Page, End Page
df1["pages"] = df1['BP'].astype(str) + '-' + df1['EP'].astype(str)
df1[df1["DI"].isna()]


<ipython-input-5-d31685ecafe3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["pages"] = df1['BP'].astype(str) + '-' + df1['EP'].astype(str)


,DI,D2,AU,TI,SO,PY,VL,IS,BP,EP,pages
2,NaN,NaN,"Jean, Peter K. B. St.; Weitzer, Ronald",Race and Policing in America: Conflict and reform,AMERICAN JOURNAL OF SOCIOLOGY,2008.0,114.0,1.0,254,256,254-256
31,NaN,NaN,"Ben-Yehuda, Nachman",The path of the devil: Early modern witch hunts,SOCIAL FORCES,2008.0,86.0,4.0,1844,1845,1844-1845
36,NaN,NaN,"Cadge, Wendy",Straight to Jesus: Sexual and Christian conver...,SOCIAL FORCES,2008.0,86.0,4.0,1841,1843,1841-1843
39,NaN,NaN,"Collins, Patricia Hill",Party/politics: Horizons in black political th...,SOCIAL FORCES,2008.0,86.0,4.0,1865,1867,1865-1867
40,NaN,NaN,"Crowley, Martha",From hire to liar: The role of deception in th...,SOCIAL FORCES,2008.0,86.0,4.0,1867,1869,1867-1869
...,...,...,...,...,...,...,...,...,...,...,...
6311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan-nan
6312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan-nan
6313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan-nan
6314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan-nan


In [6]:

df_join = df_s.merge(df1, left_on = ["year","volume", "issue", "BP"], right_on=["PY","VL","IS","BP"], how="left")
df_join



,artid,journal,volume,issue,year,month,pages_x,cited3,cited2012,aut_num,...,DI,D2,AU,TI,SO,PY,VL,IS,EP_y,pages_y
0,1,annurevisoci,35,1,2009,6,1-19,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,annurevisoci,35,1,2009,6,21-42,15,15,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,annurevisoci,35,1,2009,6,43-65,17,21,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,annurevisoci,35,1,2009,6,67-85,6,7,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,annurevisoci,35,1,2009,6,87-105,18,18,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3603,3746,amersocirevi,65,1,2000,2,122-127,3,21,3,...,10.2307/2657293,NaN,"Frank, DJ; Hironaka, A; Schofer, E",Environmentalism as a global institution -: <i...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,127,122-127
3604,3747,amersocirevi,65,1,2000,2,138-167,8,54,1,...,10.2307/2657295,NaN,"Gorski, PS",Historicizing the secularization debate: Churc...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,167,138-167
3605,3748,amersocirevi,65,1,2000,2,128-137,1,7,4,...,10.2307/2657294,NaN,"Horton, HD; Allen, BL; Herring, C; Thomas, ME",Lost in the storm: The sociology of the black ...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,137,128-137
3606,3749,amersocirevi,65,1,2000,2,96-116,16,186,3,...,10.2307/2657291,NaN,"Frank, DJ; Hironaka, A; Schofer, E",The nation-state and the natural environment o...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,116,96-116


In [7]:
df1.groupby("SO").count()

,DI,D2,AU,TI,PY,VL,IS,BP,EP,pages
SO,,,,,,,,,,
AMERICAN JOURNAL OF SOCIOLOGY,1893,0,1989,1989,1989,1989,1978,1989,1989,1989
AMERICAN SOCIOLOGICAL REVIEW,465,0,481,481,481,481,481,481,481,481
ANNUAL REVIEW OF SOCIOLOGY,233,0,234,239,239,239,0,239,239,239
JOURNAL OF MARRIAGE AND FAMILY,922,0,1100,1100,1100,1100,1100,1092,1092,1100
JOURNAL OF SUSTAINABLE AGRICULTURE,1,0,1,1,1,1,1,1,1,1
SOCIAL FORCES,945,0,1189,1189,1189,1189,1189,1187,1187,1189
SOCIAL NETWORKS,252,0,252,252,252,252,252,252,252,252
SOCIAL SCIENCE RESEARCH,439,0,443,443,443,443,443,443,443,443
"SOCIOLOGICAL METHODOLOGY 2000, VOL 30",7,0,8,8,8,8,0,8,8,8


In [8]:
na = df_join[df_join["DI"].isna()]

In [9]:
na = na.reset_index()
na

,index,artid,journal,volume,issue,year,month,pages_x,cited3,cited2012,...,DI,D2,AU,TI,SO,PY,VL,IS,EP_y,pages_y
0,0,1,annurevisoci,35,1,2009,6,1-19,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,annurevisoci,35,1,2009,6,21-42,15,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3,annurevisoci,35,1,2009,6,43-65,17,21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4,annurevisoci,35,1,2009,6,67-85,6,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5,annurevisoci,35,1,2009,6,87-105,18,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,3578,3721,socimeth,30,0,2000,6,1-28,2,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
580,3579,3722,socimeth,30,0,2000,6,237-260,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
581,3580,3723,socimeth,30,0,2000,6,261-315,0,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
582,3581,3724,socimeth,30,0,2000,6,165-202,2,73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
na_new = na.merge(df1, left_on = ["year","volume", "BP"], right_on=["PY","VL","BP"], how="left")
na_new = na_new.reset_index(drop=True)
na_new

,index,artid,journal,volume,issue,year,month,pages_x,cited3,cited2012,...,DI_y,D2_y,AU_y,TI_y,SO_y,PY_y,VL_y,IS_y,EP,pages
0,0,1,annurevisoci,35,1,2009,6,1-19,0,0,...,NaN,NaN,NaN,ANNUAL REVIEW OF SOCIOLOGY,ANNUAL REVIEW OF SOCIOLOGY,2009.0,35.0,NaN,608,1-608
1,0,1,annurevisoci,35,1,2009,6,1-19,0,0,...,10.1146/annurev-soc-070308-115936,NaN,"Gans, Herbert J.",Working in Six Research Areas: A Multi-Field S...,ANNUAL REVIEW OF SOCIOLOGY,2009.0,35.0,NaN,19,1-19
2,1,2,annurevisoci,35,1,2009,6,21-42,15,15,...,10.1146/annurev-soc-070308-115916,NaN,"Brubaker, Rogers","Ethnicity, Race, and Nationalism",ANNUAL REVIEW OF SOCIOLOGY,2009.0,35.0,NaN,42,21-42
3,2,3,annurevisoci,35,1,2009,6,43-65,17,21,...,10.1146/annurev-soc-070308-115954,NaN,"Jacobs, Jerry A.; Frickel, Scott",Interdisciplinarity: A Critical Assessment,ANNUAL REVIEW OF SOCIOLOGY,2009.0,35.0,NaN,65,43-65
4,3,4,annurevisoci,35,1,2009,6,67-85,6,7,...,10.1146/annurev.soc.34.040507.134631,NaN,"Andersen, Robert",Nonparametric Methods for Modeling Nonlinearit...,ANNUAL REVIEW OF SOCIOLOGY,2009.0,35.0,NaN,85,67-85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,3578,3721,socimeth,30,0,2000,6,1-28,2,4,...,NaN,NaN,"Arminger, G; Clogg, CC; Cheng, TW",Regression analysis of multivariate binary res...,"SOCIOLOGICAL METHODOLOGY 2000, VOL 30",2000.0,30.0,NaN,26,1-26
607,3579,3722,socimeth,30,0,2000,6,237-260,0,1,...,10.1111/0081-1750.00080,NaN,"Hoem, JM",Systematic patterns of zero exposures in event...,"SOCIOLOGICAL METHODOLOGY 2000, VOL 30",2000.0,30.0,NaN,259,237-259
608,3580,3723,socimeth,30,0,2000,6,261-315,0,12,...,10.1111/0081-1750.00081,NaN,"Montgomery, JD","The self as a fuzzy set of roles, role theory ...","SOCIOLOGICAL METHODOLOGY 2000, VOL 30",2000.0,30.0,NaN,314,261-314
609,3581,3724,socimeth,30,0,2000,6,165-202,2,73,...,10.1111/0081-1750.00078,NaN,"Yuan, KH; Bentler, PM",Three likelihood-based methods for mean and co...,"SOCIOLOGICAL METHODOLOGY 2000, VOL 30",2000.0,30.0,NaN,200,165-200


In [11]:
na_new[na_new["DI_y"].isna()]

,index,artid,journal,volume,issue,year,month,pages_x,cited3,cited2012,...,DI_y,D2_y,AU_y,TI_y,SO_y,PY_y,VL_y,IS_y,EP,pages
0,0,1,annurevisoci,35,1,2009,6,1-19,0,0,...,NaN,NaN,NaN,ANNUAL REVIEW OF SOCIOLOGY,ANNUAL REVIEW OF SOCIOLOGY,2009.0,35.0,NaN,608,1-608
28,27,28,annurevisoci,34,1,2008,6,1-13,3,3,...,NaN,NaN,NaN,ANNUAL REVIEW OF SOCIOLOGY,ANNUAL REVIEW OF SOCIOLOGY,2008.0,34.0,NaN,540,1-540
51,49,50,annurevisoci,33,1,2007,6,1-19,3,7,...,NaN,NaN,NaN,ANNUAL REVIEW OF SOCIOLOGY,ANNUAL REVIEW OF SOCIOLOGY,2007.0,33.0,NaN,593,1-593
77,74,75,annurevisoci,32,1,2006,6,1-23,0,0,...,NaN,NaN,NaN,ANNUAL REVIEW OF SOCIOLOGY,ANNUAL REVIEW OF SOCIOLOGY,2006.0,32.0,NaN,500,1-500
97,93,94,annurevisoci,31,1,2005,6,1-24,3,10,...,NaN,NaN,NaN,ANNUAL REVIEW OF SOCIOLOGY,ANNUAL REVIEW OF SOCIOLOGY,2005.0,31.0,NaN,438,1-438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,3447,3590,socsciresearch,31,3,2002,9,334-363,4,20,...,NaN,NaN,"Robison, KK; Crenshaw, EM",Post-industrial transformations and cyber-spac...,SOCIAL SCIENCE RESEARCH,2002.0,31.0,3.0,363,334-363
585,3545,3688,socialforces,79,1,2000,9,1-28,2,8,...,NaN,NaN,"Tittle, CR; Rotolo, T",IQ and stratification: An empirical evaluation...,SOCIAL FORCES,2000.0,79.0,1.0,28,1-28
586,3554,3697,socialforces,78,4,2000,6,1227-1276,2,16,...,NaN,NaN,"Rytina, S",Is occupational mobility declining in the US?,SOCIAL FORCES,2000.0,78.0,4.0,1276,1227-1276
591,3563,3706,socimeth,35,0,2005,8,1-97,17,59,...,NaN,NaN,NaN,"Sociological Methodology 2005, Vol 35","SOCIOLOGICAL METHODOLOGY 2005, VOL 35",2005.0,35.0,NaN,348,1-348


In [12]:
#na_new["DI_y"]
na_new[na_new["DI_x"].isna()]
na_new.columns

Index(['index', 'artid', 'journal', 'volume', 'issue', 'year', 'month',
       'pages_x', 'cited3', 'cited2012', 'aut_num', 'ack_num', 'com_num',
       'aut_cluster', 'ack_cluster', 'com_cluster', 'bip_cluster',
       'aut_jaccard', 'ack_jaccard', 'com_jaccard', 'aut_density',
       'ack_density', 'com_density', 'bip_density', 'aut_avgdegree',
       'ack_avgdegree', 'com_avgdegree', 'bip_avgdegree', 'aut_flowhier',
       'ack_flowhier', 'com_flowhier', 'aut_numcomp', 'ack_numcomp',
       'com_numcomp', 'bip_numcomp', 'BP', 'EP_x', 'DI_x', 'D2_x', 'AU_x',
       'TI_x', 'SO_x', 'PY_x', 'VL_x', 'IS_x', 'EP_y', 'pages_y', 'DI_y',
       'D2_y', 'AU_y', 'TI_y', 'SO_y', 'PY_y', 'VL_y', 'IS_y', 'EP', 'pages'],
      dtype='object')

In [13]:
na_new.columns


Index(['index', 'artid', 'journal', 'volume', 'issue', 'year', 'month',
       'pages_x', 'cited3', 'cited2012', 'aut_num', 'ack_num', 'com_num',
       'aut_cluster', 'ack_cluster', 'com_cluster', 'bip_cluster',
       'aut_jaccard', 'ack_jaccard', 'com_jaccard', 'aut_density',
       'ack_density', 'com_density', 'bip_density', 'aut_avgdegree',
       'ack_avgdegree', 'com_avgdegree', 'bip_avgdegree', 'aut_flowhier',
       'ack_flowhier', 'com_flowhier', 'aut_numcomp', 'ack_numcomp',
       'com_numcomp', 'bip_numcomp', 'BP', 'EP_x', 'DI_x', 'D2_x', 'AU_x',
       'TI_x', 'SO_x', 'PY_x', 'VL_x', 'IS_x', 'EP_y', 'pages_y', 'DI_y',
       'D2_y', 'AU_y', 'TI_y', 'SO_y', 'PY_y', 'VL_y', 'IS_y', 'EP', 'pages'],
      dtype='object')

In [14]:
na_join1 = na_new[['artid','DI_y', 'journal', 'volume', 'issue', 'year', 'month',
                   'cited3', 'cited2012', 'aut_num', 'ack_num', 'com_num',
       'aut_cluster', 'ack_cluster', 'com_cluster', 'bip_cluster',
       'aut_jaccard', 'ack_jaccard', 'com_jaccard', 'aut_density',
       'ack_density', 'com_density', 'bip_density', 'aut_avgdegree',
       'ack_avgdegree', 'com_avgdegree', 'bip_avgdegree', 'aut_flowhier',
       'ack_flowhier', 'com_flowhier', 'aut_numcomp', 'ack_numcomp',
       'com_numcomp', 'bip_numcomp', 'BP', 'EP_x', 'pages']]
na_join1 = na_join1.rename(columns = {'EP_x':'EP', 'DI_y':'DI'})
na_join1

,artid,DI,journal,volume,issue,year,month,cited3,cited2012,aut_num,...,aut_flowhier,ack_flowhier,com_flowhier,aut_numcomp,ack_numcomp,com_numcomp,bip_numcomp,BP,EP,pages
0,1,NaN,annurevisoci,35,1,2009,6,0,0,1,...,99.0,99.0,99.000000,1,0,1,1,1,19,1-608
1,1,10.1146/annurev-soc-070308-115936,annurevisoci,35,1,2009,6,0,0,1,...,99.0,99.0,99.000000,1,0,1,1,1,19,1-19
2,2,10.1146/annurev-soc-070308-115916,annurevisoci,35,1,2009,6,15,15,1,...,99.0,99.0,99.000000,1,0,1,1,21,42,21-42
3,3,10.1146/annurev-soc-070308-115954,annurevisoci,35,1,2009,6,17,21,2,...,1.0,0.2,0.277778,1,3,3,5,43,65,43-65
4,4,10.1146/annurev.soc.34.040507.134631,annurevisoci,35,1,2009,6,6,7,1,...,99.0,99.0,0.000000,1,2,2,2,67,85,67-85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,3721,NaN,socimeth,30,0,2000,6,2,4,3,...,99.0,99.0,99.000000,3,0,3,3,1,28,1-26
607,3722,10.1111/0081-1750.00080,socimeth,30,0,2000,6,0,1,1,...,99.0,99.0,99.000000,1,0,1,1,237,260,237-259
608,3723,10.1111/0081-1750.00081,socimeth,30,0,2000,6,0,12,1,...,99.0,99.0,99.000000,1,2,3,3,261,315,261-314
609,3724,10.1111/0081-1750.00078,socimeth,30,0,2000,6,2,73,2,...,0.0,99.0,0.000000,1,0,1,2,165,202,165-200


In [15]:
ya = df_join[(df_join["DI"].isna()==False) & (df_join["pages_x"].isna()==False)]
ya1 = ya[['artid', 'DI', 'journal', 'volume', 'issue', 'year', 'month', 'pages_x',
       'cited3', 'cited2012', 'aut_num', 'ack_num', 'com_num', 'aut_cluster',
       'ack_cluster', 'com_cluster', 'bip_cluster', 'aut_jaccard',
       'ack_jaccard', 'com_jaccard', 'aut_density', 'ack_density',
       'com_density', 'bip_density', 'aut_avgdegree', 'ack_avgdegree',
       'com_avgdegree', 'bip_avgdegree', 'aut_flowhier', 'ack_flowhier',
       'com_flowhier', 'aut_numcomp', 'ack_numcomp', 'com_numcomp',
       'bip_numcomp', 'BP', 'EP_x']]
ya1 = ya1.rename(columns={'pages_x':'pages', 'EP_x':'EP'})
ya1

,artid,DI,journal,volume,issue,year,month,pages,cited3,cited2012,...,bip_avgdegree,aut_flowhier,ack_flowhier,com_flowhier,aut_numcomp,ack_numcomp,com_numcomp,bip_numcomp,BP,EP
233,234,10.1177/000312240907400505,amersocirevi,74,5,2009,10,777-799,28,29,...,0.000000,1.0,99.000000,1.000000,1,1,2,4,777,799
234,235,10.1177/000312240907400504,amersocirevi,74,5,2009,10,756-776,12,12,...,0.000000,99.0,1.000000,1.000000,1,3,4,6,756,776
235,236,10.1177/000312240907400506,amersocirevi,74,5,2009,10,800-820,15,15,...,1.875000,0.0,0.333333,0.086957,1,4,2,3,800,820
236,237,10.1177/000312240907400502,amersocirevi,74,5,2009,10,709-730,16,16,...,0.000000,99.0,99.000000,99.000000,1,0,1,1,709,730
237,238,10.1177/000312240907400501,amersocirevi,74,5,2009,10,683-708,7,8,...,0.833333,99.0,0.250000,0.111111,1,3,3,4,683,708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3603,3746,10.2307/2657293,amersocirevi,65,1,2000,2,122-127,3,21,...,4.250000,0.0,99.000000,0.000000,1,1,1,1,122,127
3604,3747,10.2307/2657295,amersocirevi,65,1,2000,2,138-167,8,54,...,0.333333,99.0,99.000000,0.000000,1,5,5,5,138,167
3605,3748,10.2307/2657294,amersocirevi,65,1,2000,2,128-137,1,7,...,0.000000,1.0,99.000000,1.000000,3,1,4,5,128,137
3606,3749,10.2307/2657291,amersocirevi,65,1,2000,2,96-116,16,186,...,4.250000,0.0,99.000000,0.000000,1,1,1,1,96,116


In [16]:
df3 = pd.concat([ya1,na_join1])
df3 = df3.reset_index(drop=True)
df3

,artid,DI,journal,volume,issue,year,month,pages,cited3,cited2012,...,bip_avgdegree,aut_flowhier,ack_flowhier,com_flowhier,aut_numcomp,ack_numcomp,com_numcomp,bip_numcomp,BP,EP
0,234,10.1177/000312240907400505,amersocirevi,74,5,2009,10,777-799,28,29,...,0.000000,1.0,99.000000,1.000000,1,1,2,4,777,799
1,235,10.1177/000312240907400504,amersocirevi,74,5,2009,10,756-776,12,12,...,0.000000,99.0,1.000000,1.000000,1,3,4,6,756,776
2,236,10.1177/000312240907400506,amersocirevi,74,5,2009,10,800-820,15,15,...,1.875000,0.0,0.333333,0.086957,1,4,2,3,800,820
3,237,10.1177/000312240907400502,amersocirevi,74,5,2009,10,709-730,16,16,...,0.000000,99.0,99.000000,99.000000,1,0,1,1,709,730
4,238,10.1177/000312240907400501,amersocirevi,74,5,2009,10,683-708,7,8,...,0.833333,99.0,0.250000,0.111111,1,3,3,4,683,708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3630,3721,NaN,socimeth,30,0,2000,6,1-26,2,4,...,0.000000,99.0,99.000000,99.000000,3,0,3,3,1,28
3631,3722,10.1111/0081-1750.00080,socimeth,30,0,2000,6,237-259,0,1,...,0.000000,99.0,99.000000,99.000000,1,0,1,1,237,260
3632,3723,10.1111/0081-1750.00081,socimeth,30,0,2000,6,261-314,0,12,...,0.000000,99.0,99.000000,99.000000,1,2,3,3,261,315
3633,3724,10.1111/0081-1750.00078,socimeth,30,0,2000,6,165-200,2,73,...,0.000000,0.0,99.000000,0.000000,1,0,1,2,165,202


In [17]:
gb1 = df3.groupby('artid').count()
gb2 = gb1[gb1["journal"]>1].reset_index(drop=False)
gb2

#duplicates
temp = df3.merge(gb2[["artid","DI"]], on="artid", how = "left").reset_index(drop=True)
temp1 = temp[temp["DI_y"].isna()==False]
temp1 = temp1.sort_values(by='artid', ascending=False)
temp2 = temp1[temp1["DI_x"].isna()==False]
temp3 = temp2.drop_duplicates(keep='first').reset_index(drop=True)
temp3

db2 = gb1[gb1["journal"]==1].reset_index(drop=False)
demp = df3.merge(db2[["artid","DI"]], on="artid", how = "left").reset_index(drop=True)

demp1 = demp[demp["DI_y"].isna()==False]
demp1 = demp1.sort_values(by='artid', ascending=False)
demp2 = demp1[demp1["DI_x"].isna()==False]
demp3 = demp2.drop_duplicates(keep='first').reset_index(drop=True)

df4 = pd.concat([demp3, temp3]).sort_values(by='artid', ascending=True).drop("DI_y", axis=1).reset_index(drop=True)
df4 = df4.rename(columns={"DI_x":"DI"})
print(len(df3), len(temp3), len(db2), len(demp3), len(temp3)+len(demp3))
df4

3635 11 3578 3344 3355


,artid,DI,journal,volume,issue,year,month,pages,cited3,cited2012,...,bip_avgdegree,aut_flowhier,ack_flowhier,com_flowhier,aut_numcomp,ack_numcomp,com_numcomp,bip_numcomp,BP,EP
0,1,10.1146/annurev-soc-070308-115936,annurevisoci,35,1,2009,6,1-19,0,0,...,0.000000,99.0,99.0,99.000000,1,0,1,1,1,19
1,2,10.1146/annurev-soc-070308-115916,annurevisoci,35,1,2009,6,21-42,15,15,...,0.000000,99.0,99.0,99.000000,1,0,1,1,21,42
2,3,10.1146/annurev-soc-070308-115954,annurevisoci,35,1,2009,6,43-65,17,21,...,0.777778,1.0,0.2,0.277778,1,3,3,5,43,65
3,4,10.1146/annurev.soc.34.040507.134631,annurevisoci,35,1,2009,6,67-85,6,7,...,0.666667,99.0,99.0,0.000000,1,2,2,2,67,85
4,5,10.1146/annurev-soc-070308-115920,annurevisoci,35,1,2009,6,87-105,18,18,...,0.000000,0.0,99.0,0.000000,1,0,1,2,87,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3350,3746,10.2307/2657293,amersocirevi,65,1,2000,2,122-127,3,21,...,4.250000,0.0,99.0,0.000000,1,1,1,1,122,127
3351,3747,10.2307/2657295,amersocirevi,65,1,2000,2,138-167,8,54,...,0.333333,99.0,99.0,0.000000,1,5,5,5,138,167
3352,3748,10.2307/2657294,amersocirevi,65,1,2000,2,128-137,1,7,...,0.000000,1.0,99.0,1.000000,3,1,4,5,128,137
3353,3749,10.2307/2657291,amersocirevi,65,1,2000,2,96-116,16,186,...,4.250000,0.0,99.0,0.000000,1,1,1,1,96,116


In [18]:
df_join["pages_diff"] = df_join['pages_y'] != df_join['pages_x']


df_join.loc[(df_join['pages_diff']) & (df_join['pages_y'].isna()), 'pages_y'] = df_join['pages_x']

df_join['pages_diff'] = df_join['pages_y'] != df_join['pages_x']

df_join



,artid,journal,volume,issue,year,month,pages_x,cited3,cited2012,aut_num,...,D2,AU,TI,SO,PY,VL,IS,EP_y,pages_y,pages_diff
0,1,annurevisoci,35,1,2009,6,1-19,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-19,False
1,2,annurevisoci,35,1,2009,6,21-42,15,15,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21-42,False
2,3,annurevisoci,35,1,2009,6,43-65,17,21,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43-65,False
3,4,annurevisoci,35,1,2009,6,67-85,6,7,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67-85,False
4,5,annurevisoci,35,1,2009,6,87-105,18,18,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87-105,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3603,3746,amersocirevi,65,1,2000,2,122-127,3,21,3,...,NaN,"Frank, DJ; Hironaka, A; Schofer, E",Environmentalism as a global institution -: <i...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,127,122-127,False
3604,3747,amersocirevi,65,1,2000,2,138-167,8,54,1,...,NaN,"Gorski, PS",Historicizing the secularization debate: Churc...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,167,138-167,False
3605,3748,amersocirevi,65,1,2000,2,128-137,1,7,4,...,NaN,"Horton, HD; Allen, BL; Herring, C; Thomas, ME",Lost in the storm: The sociology of the black ...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,137,128-137,False
3606,3749,amersocirevi,65,1,2000,2,96-116,16,186,3,...,NaN,"Frank, DJ; Hironaka, A; Schofer, E",The nation-state and the natural environment o...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,116,96-116,False


In [19]:
df_join[df_join["DI"].isna()]


,artid,journal,volume,issue,year,month,pages_x,cited3,cited2012,aut_num,...,D2,AU,TI,SO,PY,VL,IS,EP_y,pages_y,pages_diff
0,1,annurevisoci,35,1,2009,6,1-19,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-19,False
1,2,annurevisoci,35,1,2009,6,21-42,15,15,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21-42,False
2,3,annurevisoci,35,1,2009,6,43-65,17,21,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43-65,False
3,4,annurevisoci,35,1,2009,6,67-85,6,7,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67-85,False
4,5,annurevisoci,35,1,2009,6,87-105,18,18,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87-105,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3578,3721,socimeth,30,0,2000,6,1-28,2,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-28,False
3579,3722,socimeth,30,0,2000,6,237-260,0,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,237-260,False
3580,3723,socimeth,30,0,2000,6,261-315,0,12,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,261-315,False
3581,3724,socimeth,30,0,2000,6,165-202,2,73,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,165-202,False


In [20]:
df_join_updated = df_join.merge(
    na_new[['year', 'volume', 'BP', 'DI_y']],  
    on=['year', 'volume', 'BP'],            
    how='left',                              
    suffixes=('', '_new')                   
)

df_join_updated['DI'] = df_join_updated['DI'].fillna(df_join_updated['DI_y'])

df_join_updated = df_join_updated.drop(columns=['DI_y'])

num_missing_doi_after_update = df_join_updated['DI'].isna().sum()
print(f"Total articles still missing DOI after update: {num_missing_doi_after_update}")

df_join_updated

Total articles still missing DOI after update: 311


,artid,journal,volume,issue,year,month,pages_x,cited3,cited2012,aut_num,...,D2,AU,TI,SO,PY,VL,IS,EP_y,pages_y,pages_diff
0,1,annurevisoci,35,1,2009,6,1-19,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-19,False
1,1,annurevisoci,35,1,2009,6,1-19,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-19,False
2,2,annurevisoci,35,1,2009,6,21-42,15,15,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21-42,False
3,3,annurevisoci,35,1,2009,6,43-65,17,21,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43-65,False
4,4,annurevisoci,35,1,2009,6,67-85,6,7,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67-85,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3663,3746,amersocirevi,65,1,2000,2,122-127,3,21,3,...,NaN,"Frank, DJ; Hironaka, A; Schofer, E",Environmentalism as a global institution -: <i...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,127,122-127,False
3664,3747,amersocirevi,65,1,2000,2,138-167,8,54,1,...,NaN,"Gorski, PS",Historicizing the secularization debate: Churc...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,167,138-167,False
3665,3748,amersocirevi,65,1,2000,2,128-137,1,7,4,...,NaN,"Horton, HD; Allen, BL; Herring, C; Thomas, ME",Lost in the storm: The sociology of the black ...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,137,128-137,False
3666,3749,amersocirevi,65,1,2000,2,96-116,16,186,3,...,NaN,"Frank, DJ; Hironaka, A; Schofer, E",The nation-state and the natural environment o...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,116,96-116,False


In [21]:
df_join_updated = df_join_updated.drop_duplicates(subset=["year", "volume", "journal", "pages_x", "DI"], keep="first")

df_join_updated

,artid,journal,volume,issue,year,month,pages_x,cited3,cited2012,aut_num,...,D2,AU,TI,SO,PY,VL,IS,EP_y,pages_y,pages_diff
0,1,annurevisoci,35,1,2009,6,1-19,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-19,False
1,1,annurevisoci,35,1,2009,6,1-19,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-19,False
2,2,annurevisoci,35,1,2009,6,21-42,15,15,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21-42,False
3,3,annurevisoci,35,1,2009,6,43-65,17,21,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43-65,False
4,4,annurevisoci,35,1,2009,6,67-85,6,7,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67-85,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3663,3746,amersocirevi,65,1,2000,2,122-127,3,21,3,...,NaN,"Frank, DJ; Hironaka, A; Schofer, E",Environmentalism as a global institution -: <i...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,127,122-127,False
3664,3747,amersocirevi,65,1,2000,2,138-167,8,54,1,...,NaN,"Gorski, PS",Historicizing the secularization debate: Churc...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,167,138-167,False
3665,3748,amersocirevi,65,1,2000,2,128-137,1,7,4,...,NaN,"Horton, HD; Allen, BL; Herring, C; Thomas, ME",Lost in the storm: The sociology of the black ...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,137,128-137,False
3666,3749,amersocirevi,65,1,2000,2,96-116,16,186,3,...,NaN,"Frank, DJ; Hironaka, A; Schofer, E",The nation-state and the natural environment o...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,116,96-116,False


In [22]:
df_join_updated[df_join_updated["DI"].isna()]

,artid,journal,volume,issue,year,month,pages_x,cited3,cited2012,aut_num,...,D2,AU,TI,SO,PY,VL,IS,EP_y,pages_y,pages_diff
0,1,annurevisoci,35,1,2009,6,1-19,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-19,False
28,28,annurevisoci,34,1,2008,6,1-13,3,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-13,False
51,50,annurevisoci,33,1,2007,6,1-19,3,7,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-19,False
77,75,annurevisoci,32,1,2006,6,1-23,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-23,False
97,94,annurevisoci,31,1,2005,6,1-24,3,10,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-24,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3506,3590,socsciresearch,31,3,2002,9,334-363,4,20,2,...,NaN,"Robison, KK; Crenshaw, EM",Post-industrial transformations and cyber-spac...,SOCIAL SCIENCE RESEARCH,2002.0,31.0,3.0,363,334-363,False
3604,3688,socialforces,79,1,2000,9,1-28,2,8,2,...,NaN,"Tittle, CR; Rotolo, T",IQ and stratification: An empirical evaluation...,SOCIAL FORCES,2000.0,79.0,1.0,28,1-28,False
3613,3697,socialforces,78,4,2000,6,1227-1276,2,16,1,...,NaN,"Rytina, S",Is occupational mobility declining in the US?,SOCIAL FORCES,2000.0,78.0,4.0,1276,1227-1276,False
3623,3706,socimeth,35,0,2005,8,1-97,17,59,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-97,False


In [23]:

# Manually mapped DOIs for missing articles
manual_doi_mapping = {
    ("annurevisoci", 34, 2008, '1'): "10.1146/annurev.soc.34.040507.134654",
    ("annurevisoci", 33, 2007, '1'): "10.1146/annurev.soc.33.040406.131720",
    ('annurevisoci', 32, 2006, '1'): "10.1146/annurev.soc.32.061604.123051",
    ('annurevisoci', 31, 2005, '1'): "10.1146/annurev.soc.31.041304.122209",
    ('amersocirevi', 67, 2002, '389'): "10.2307/3088963",
    ('amersocirevi', 69, 2004, '1'): "10.1177/000312240406900102",
    ('amersocirevi', 70, 2005, '136'): "10.2307/4145353",
    ('socnetworks', 28, 2006, '97'): "10.1016/j.socnet.2005.04.005",
    ('socialforces', 86, 2008, '1753'): "10.1353/sof.0.0038",
    ('socialforces', 86, 2008, '1781'): "10.1353/sof.0.0043",
    ('socialforces', 86, 2008, '1217'): "10.1353/sof.0.0008",
    ('socialforces', 86, 2008, '1189'): "10.1353/sof.0.0004",
    ('socialforces', 86, 2007, '417'): "10.1353/sof.2008.0022",
    ('socialforces', 86, 2008, '1481'): "10.1353/sof.0.0057",
    ('socialforces', 86, 2008, '1561'): "10.1353/sof.0.0040",
    ('socialforces', 86, 2008, '1529'): "10.1353/sof.0.0035",
    ('socialforces', 86, 2008, '1455'): "10.1353/sof.0.0052",
    ('socialforces', 86, 2008, '1367'): "10.1353/sof.0.0037",
    ('socialforces', 86, 2008, '1675'): "10.1353/sof.0.0059",
    ('socialforces', 86, 2008, '1805'): "10.1353/sof.0.0048",
    ('socialforces', 86, 2008, '1848'): "10.1353/sof.0.0056",
    ('socialforces', 86, 2008, '1273'): "10.1353/sof.0.0018",
    ('socialforces', 86, 2008, '995'): "10.1353/sof.0.0002",
    ('socialforces', 86, 2008, '911'): "10.1353/sof.0.0015",
    ('socialforces', 86, 2008, '1027'): "10.1353/sof.0.0027",
    ('socialforces', 86, 2008, '1344'): "10.1353/sof.0.0011",

    ('socialforces', 86, 2008, '937'): "10.1353/sof.0.0010",
    ('socialforces', 86, 2008, '1079'): "10.1353/sof.0.0017",
    ('socialforces', 84, 2005, '595'): "10.1353/sof.2005.0095",
    ('socialforces', 84, 2006, '1455'): "10.1353/sof.2006.0072",
    ('socialforces', 85, 2006, '145'): "10.1353/sof.2006.0126",
    ('socialforces', 83, 2005, '1065'): "10.1353/sof.2005.0040",
    ('social forces', 83, 2005, 1443): "",
    ('socialforces', 83, 2005, '1697'): "10.1353/sof.2005.0060",
    ('jmarriagefamily', 70, 2008, '466'): "10.1111/j.1741-3737.2008.00494.x",
    ('jmarriagefamily', 69, 2007, '1107'): "10.1111/j.1741-3737.2007.00435.x",
    ('jmarriagefamily', 68, 2006, '56'): "10.1111/j.1741-3737.2006.00233.x",
    ('jmarriagefamily', 67, 2005, '352'): "10.1111/j.0022-2445.2005.00121.x",
    ('jmarriagefamily', 67, 2005, '1'): "10.1111/j.0022-2445.2005.00001.x",
    ('jmarriagefamily', 67, 2005, '627'): "10.1111/j.1741-3737.2005.00158.x",
    ('jmarriagefamily', 67, 2005, '85'): "10.1111/j.0022-2445.2005.00007.x",
    ('jmarriagefamily', 66, 2004, '1042'): "10.1111/j.0022-2445.2004.00077.x",
    ('jmarriagefamily', 66, 2004, '821'): "10.1111/j.0022-2445.2004.00055.x",
    ('jmarriagefamily', 66, 2004, '1332'): "10.1111/j.0022-2445.2004.00096.x",
    ('jmarriagefamily', 66, 2004, '1300'): "10.1111/j.0022-2445.2004.00094.x",
    ('jmarriagefamily', 66, 2004, '1231'): "10.1111/j.0022-2445.2004.00089.x",
    ('jmarriagefamily', 66, 2004, '210'): "10.1111/j.0022-2445.2004.00014.x-i1",
    ('jmarriagefamily', 66, 2004, '318'): "10.1111/j.1741-3737.2004.00023.x",
    ('jmarriagefamily', 66, 2004, '515'): "10.1111/j.1741-3737.2004.00035.x",
    ('jmarriagefamily', 66, 2004, '163'): "10.1111/j.0022-2445.2004.00011.x-i1",
    ('jmarriagefamily', 66, 2004, '101'): "10.1111/1467-6478.00050-i1",
    ('jmarriagefamily', 66, 2004, '236'): "10.1111/j.0022-2445.2004.00015.x",
    ('jmarriagefamily', 65, 2003, '1088'): "10.1111/j.1741-3737.2003.01081.x",
    ('jmarriagefamily', 65, 2003, '1082'): "10.1111/j.1741-3737.2003.01081.x",
    ('jmarriagefamily', 65, 2003, '1081'): "10.1111/j.1741-3737.2003.01081.x",
    ('jmarriagefamily', 65, 2003, '1085'): "10.1111/j.1741-3737.2003.01081.x",
    ('jmarriagefamily', 65, 2003, '1087'): "10.1111/j.1741-3737.2003.01081.x",
    ('jmarriagefamily', 65, 2003, '1084'): "10.1111/j.1741-3737.2003.01081.x",
    ('jmarriagefamily', 65, 2003, '1083'): "10.1111/j.1741-3737.2003.01081.x",
    ('jmarriagefamily', 65, 2003, '767'): "10.1111/j.1741-3737.2003.00762.x",
    ('jmarriagefamily', 65, 2003, '764'): "10.1111/j.1741-3737.2003.00762.x",
    ('jmarriagefamily', 65, 2003, '766'): "10.1111/j.1741-3737.2003.00762.x",
    ('jmarriagefamily', 65, 2003, '763'): "10.1111/j.1741-3737.2003.00762.x",
    ('jmarriagefamily', 65, 2003, '765'): "10.1111/j.1741-3737.2003.00762.x",
    ('jmarriagefamily', 65, 2003, '768'): "10.1111/j.1741-3737.2003.00762.x",
    ('jmarriagefamily', 65, 2003, '762'): "10.1111/j.1741-3737.2003.00762.x",
    ('jmarriagefamily', 65, 2003, '501'): "10.1111/j.1741-3737.2003.00496.x",
    ('jmarriagefamily', 65, 2003, '500'): "10.1111/j.1741-3737.2003.00496.x",
    ('jmarriagefamily', 65, 2003, '502'): "10.1111/j.1741-3737.2003.00496.x",
    ('jmarriagefamily', 65, 2003, '504'): "10.1111/j.1741-3737.2003.00496.x",
    ('jmarriagefamily', 65, 2003, '1089'): "10.1111/j.1741-3737.2003.01081.x",
    ('jmarriagefamily', 65, 2003, '769'): "10.1111/j.1741-3737.2003.00762.x",
    ('jmarriagefamily', 65, 2003, '496'): "10.1111/j.1741-3737.2003.00496.x",
    ('jmarriagefamily', 65, 2003, '499'): "10.1111/j.1741-3737.2003.00496.x",
    ('jmarriagefamily', 65, 2003, '503'): "10.1111/j.1741-3737.2003.00496.x",
    ('jmarriagefamily', 65, 2003, '498'): "10.1111/j.1741-3737.2003.00496.x",
    ('jmarriagefamily', 64, 2002, '270'): "10.1111/j.1741-3737.2002.00269.x",
    ('jmarriagefamily', 63, 2001, '804'): "10.1111/j.1741-3737.2001.00804.x",
    ('jmarriagefamily', 63, 2001, '582'): "10.1111/j.1741-3737.2001.00580.x",
    ('jmarriagefamily', 63, 2001, '589'): "10.1111/j.1741-3737.2001.00580.x",
    ('jmarriagefamily', 63, 2001, '592'): "10.1111/j.1741-3737.2001.00580.x",
    ('jmarriagefamily', 63, 2001, '585'): "10.1111/j.1741-3737.2001.00580.x",
    ('jmarriagefamily', 63, 2001, '279'): "10.1111/j.1741-3737.2001.00276.x",
    ('jmarriagefamily', 63, 2001, '1206'): "10.1111/j.1741-3737.2001.01199.x",
    ('jmarriagefamily', 63, 2001, '903'): "10.1111/j.1741-3737.2001.00902.x",
    ('socitheo', 22, 2004, '1'): "10.1111/j.1467-9558.2004.00199.x",
    ('socieduc', 80, 2007, '43'): "10.1177/0038040707080001",
    ('socieduc', 79, 2006, '1'): "10.1177/003804070607900101",
    ('socieduc', 78, 2005, '122'): "10.1177/003804070507800202",
    ('socieduc', 78, 2005, '1'): "10.1177/00380407050780010",
    
    ('socimeth', 36, 2006, '1'): "10.1111/j.1467-9531.2006.00172.x",
    ('socimeth', 37, 2007, '1'): "10.1111/j.1467-9531.2007.00184.x",
    ('socsciresearch', 32, 2003, '61'): "10.1016/S0049-089X(02)00013-3",
    ('socsciresearch', 32, 2003, '98'): "10.1016/S0049-089X(02)00021-2",
    ('socsciresearch', 31, 2002, '334'): "10.1016/S0049-089X(02)00004-2",

    ('socimeth', 35, 2005, '1'): "10.1111/j.0081-1750.2006.00164.x",
    ('socimeth', 30, 2000, '1'): "10.1111/0081-1750.00074",
    
    ('annurevisoci', 35, 2009, '1'): "10.1146/annurev-soc-070308-115936", 

    ('socialforces', 88, 2009, '945'): "10.1353/sof.0.0279", 
    ('socialforces', 88, 2009, '643'): "10.1353/sof.0.0262", 
    ('socialforces', 88, 2009, '785'): "10.1353/sof.0.0255", 
    ('socialforces', 88, 2009, '543'): "10.1353/sof.0.0278", 
    ('socialforces', 88, 2009, '723'): "10.1353/sof.0.0276", 
    ('socialforces', 88, 2009, '175'): "10.1353/sof.0.0231", 
    ('socialforces', 88, 2009, '147'): "10.1353/sof.0.0227", 
    ('socialforces', 88, 2009, '373'): "10.1353/sof.0.0234", 
    ('socialforces', 88, 2009, '425'): "10.1353/sof.0.0244", 
    ('socialforces', 88, 2009, '227'): "10.1353/sof.0.0241", 
    ('socialforces', 88, 2009, '519'): "10.1353/sof.0.0273", 
    ('socialforces', 88, 2009, '573'): "10.1353/sof.0.0252",
    ('socialforces', 88, 2009, '607'): "10.1353/sof.0.0257",
    ('socialforces', 88, 2009, '693'): "10.1353/sof.0.0271",
    ('socialforces', 88, 2009, '819'): "10.1353/sof.0.0260",
    ('socialforces', 88, 2009, '893'): "10.1353/sof.0.0269",
    ('socialforces', 88, 2009, '971'): "10.1353/sof.0.0253",
    ('socialforces', 88, 2009, '337'): "10.1353/sof.0.0229",
    ('socialforces', 88, 2009, '1'): "10.1353/sof.0.0233",
    ('socialforces', 88, 2009, '61'): "10.1353/sof.0.0243",
    ('socialforces', 88, 2009, '271'): "10.1353/sof.0.0221",
    ('socialforces', 88, 2009, '257'): "10.1353/sof.0.0246",
    ('socialforces', 88, 2009, '121'): "10.1353/sof.0.0223",

    ('socialforces', 87, 2009, '1943'): "10.1353/sof.0.0192", 
    ('socialforces', 87, 2009, '1499'): "10.1353/sof.0.0170", 
    ('socialforces', 87, 2009, '1481'): "10.1353/sof.0.0161", 
    ('socialforces', 87, 2009, '1355'): "10.1353/sof.0.0171", 
    ('socialforces', 87, 2009, '1649'): "10.1353/sof.0.0164", 
    ('socialforces', 87, 2009, '1873'): "10.1353/sof.0.0187",
    ('socialforces', 87, 2009, '2155'): "10.1353/sof.0.0218",
    ('socialforces', 87, 2009, '1743'): "10.1353/sof.0.0190",
    ('socialforces', 87, 2009, '1973'): "10.1353/sof.0.0191",
    ('socialforces', 87, 2009, '1913'): "10.1353/sof.0.0193",
    ('socialforces', 87, 2009, '2125'): "10.1353/sof.0.0219",
    ('socialforces', 87, 2009, '1997'): "10.1353/sof.0.0220",
    ('socialforces', 87, 2009, '2031'): "10.1353/sof.0.0189",
    ('socialforces', 87, 2009, '1321'): "10.1353/sof.0.0167",
    ('socialforces', 87, 2009, '1703'): "10.1353/sof.0.0162",
    ('socialforces', 87, 2009, '1529'): "10.1353/sof.0.0174",
    ('socialforces', 87, 2009, '1449'): "10.1353/sof.0.0157",
    ('socialforces', 87, 2009, '1489'): "10.1353/sof.0.0165",
    ('socialforces', 87, 2009, '1223'): "10.1353/sof.0.0177",
    ('socialforces', 87, 2009, '1389'): "10.1353/sof.0.0175",
    ('socialforces', 87, 2009, '1167'): "10.1353/sof.0.0169",
    ('socialforces', 87, 2008, '937'): "10.1353/sof.0.0138", 
    ('socialforces', 87, 2008, '1063'): "10.1353/sof.0.0136", 
    ('socialforces', 87, 2008, '741'): "10.1353/sof.0.0135", 
    ('socialforces', 87, 2008, '1158'): "10.1353/sof.0.0137", 
    ('socialforces', 87, 2008, '65'): "10.1353/sof.0.0095", 
    ('socialforces', 87, 2008, '299'): "10.1353/sof.0.0097", 
    ('socialforces', 87, 2008, '983'): "10.1353/sof.0.0149",
    ('socialforces', 87, 2008, '993'): "10.1353/sof.0.0155",
    ('socialforces', 87, 2008, '911'): "10.1353/sof.0.0133",
    ('socialforces', 87, 2008, '767'): "10.1353/sof.0.0140",
    ('socialforces', 87, 2008, '1039'): "10.1353/sof.0.0131",
    ('socialforces', 87, 2008, '887'): "10.1353/sof.0.0128",
    ('socialforces', 87, 2008, '713'): "10.1353/sof.0.0130",
    ('socialforces', 87, 2008, '795'): "10.1353/sof.0.0145",
    ('socialforces', 87, 2008, '1015'): "10.1353/sof.0.0126",
    ('socialforces', 87, 2008, '1089'): "10.1353/sof.0.0141",
    ('socialforces', 87, 2008, '1115'): "10.1353/sof.0.0146",
    ('socialforces', 87, 2008, '857'): "10.1353/sof.0.0123",
    ('socialforces', 87, 2008, '961'): "10.1353/sof.0.0143",
    ('socialforces', 87, 2008, '623'): "10.1353/sof.0.0147",
    ('socialforces', 87, 2008, '679'): "10.1353/sof.0.0125",
    ('socialforces', 87, 2008, '651'): "10.1353/sof.0.0153",
    ('socialforces', 87, 2008, '153'): "10.1353/sof.0.0065",
    ('socialforces', 87, 2008, '185'): "10.1353/sof.0.0074",
    ('socialforces', 87, 2008, '33'): "10.1353/sof.0.0086",
    ('socialforces', 87, 2008, '473'): "10.1353/sof.0.0076",
    ('socialforces', 87, 2008, '121'): "10.1353/sof.0.0104",
    ('socialforces', 87, 2008, '561'): "10.1353/sof.0.0088",
    ('socialforces', 87, 2008, '445'): "10.1353/sof.0.0068",
    ('socialforces', 87, 2008, '501'): "10.1353/sof.0.0122",
    ('socialforces', 87, 2008, '387'): "10.1353/sof.0.0079",
    
    ('socialforces', 79, 2000, '1'): "10.1093/sf/79.1.1",
    ('socialforces', 78, 2000, '1227'): "10.1093/sf/78.4.1195",
    ('socialforces', 78, 2000, '1133'): "10.1093/sf/78.3.1133",
    ('socialforces', 83, 2005, '1443'): "10.1353/sof.2005.0053", 
    ('socialforces', 86, 2007, '553'): "10.1093/sf/86.2.553", 

    ('jmarriagefamily', 62, 2000, '1018'): "10.1111/j.1741-3737.2000.01018.x",
    ('jmarriagefamily', 62, 2000, '1192'): "10.1111/j.1741-3737.2000.01192.x",
    ('jmarriagefamily', 62, 2000, '1247'): "10.1111/j.1741-3737.2000.01247.x",
    ('jmarriagefamily', 62, 2000, '911'): "10.1111/j.1741-3737.2000.00911.x",
    ('jmarriagefamily', 62, 2000, '1094'): "10.1111/j.1741-3737.2000.01094.x",
    ('jmarriagefamily', 62, 2000, '864'): "10.1111/j.1741-3737.2000.00847.x",
    ('jmarriagefamily', 62, 2000, '363'): "10.1111/j.1741-3737.2000.00363.x",
    ('jmarriagefamily', 62, 2000, '416'): "10.1111/j.1741-3737.2000.00416.x",
    ('jmarriagefamily', 62, 2000, '477'): "10.1111/j.1741-3737.2000.00477.x",
    ('jmarriagefamily', 62, 2000, '111'): "10.1111/j.1741-3737.2000.00111.x",
    ('jmarriagefamily', 62, 2000, '78'): "10.1111/j.1741-3737.2000.00078.x",
    ('jmarriagefamily', 62, 2000, '228'): "10.1111/j.1741-3737.2000.00228.x",
    ('jmarriagefamily', 70, 2008, '640'): "10.1111/j.1741-3737.2000.00640.x",
    ('jmarriagefamily', 62, 2000, '1136'): "10.1111/j.1741-3737.2000.01136.x",
    ('jmarriagefamily', 62, 2000, '1288'): "10.1111/j.1741-3737.2000.01288.x",
    ('jmarriagefamily', 62, 2000, '1208'): "10.1111/j.1741-3737.2000.01208.x",
    ('jmarriagefamily', 62, 2000, '1234'): "10.1111/j.1741-3737.2000.01234.x",
    ('jmarriagefamily', 62, 2000, '896'): "10.1111/j.1741-3737.2000.00896.x",
    ('jmarriagefamily', 62, 2000, '849'): "10.1111/j.1741-3737.2000.00847.x",
    ('jmarriagefamily', 62, 2000, '577'): "10.1111/j.1741-3737.2000.00575.x",
    ('jmarriagefamily', 62, 2000, '322'): "10.1111/j.1741-3737.2000.00322.x",
    ('jmarriagefamily', 62, 2000, '399'): "10.1111/j.1741-3737.2000.00399.x",
    ('jmarriagefamily', 62, 2000, '520'): "10.1111/j.1741-3737.2000.00520.x",
    ('jmarriagefamily', 62, 2000, '349'): "10.1111/j.1741-3737.2000.00349.x",
    ('jmarriagefamily', 62, 2000, '508'): "10.1111/j.1741-3737.2000.00508.x",
    ('jmarriagefamily', 62, 2000, '283'): "10.1111/j.1741-3737.2000.00283.x",
    ('jmarriagefamily', 62, 2000, '489'): "10.1111/j.1741-3737.2000.00489.x",
    ('jmarriagefamily', 62, 2000, '465'): "10.1111/j.1741-3737.2000.00465.x",
    ('jmarriagefamily', 62, 2000, '388'): "10.1111/j.1741-3737.2000.00388.x",
    ('jmarriagefamily', 62, 2000, '449'): "10.1111/j.1741-3737.2000.00449.x",
    ('jmarriagefamily', 62, 2000, '430'): "10.1111/j.1741-3737.2000.00430.x",
    ('jmarriagefamily', 62, 2000, '582'): "10.1111/j.1741-3737.2000.00575.x",
    ('jmarriagefamily', 62, 2000, '375'): "10.1111/j.1741-3737.2000.00375.x",
    ('jmarriagefamily', 62, 2000, '585'): "10.1111/j.1741-3737.2000.00575.x",
    ('jmarriagefamily', 62, 2000, '588'): "10.1111/j.1741-3737.2000.00575.x",
    ('jmarriagefamily', 62, 2000, '592'): "10.1111/j.1741-3737.2000.00575.x",
    ('jmarriagefamily', 62, 2000, '570'): "10.1111/j.1741-3737.2000.00570.x",
    ('jmarriagefamily', 62, 2000, '563'): "10.1111/j.1741-3737.2000.00563.x",
    ('jmarriagefamily', 62, 2000, '93'): "10.1111/j.1741-3737.2000.00093.x",
    ('jmarriagefamily', 62, 2000, '123'): "10.1111/j.1741-3737.2000.00123.x",
    ('jmarriagefamily', 62, 2000, '61'): "10.1111/j.1741-3737.2000.00061.x",
    ('jmarriagefamily', 62, 2000, '48'): "10.1111/j.1741-3737.2000.00048.x",
    ('jmarriagefamily', 62, 2000, '256'): "10.1111/j.1741-3737.2000.00256.x",
    ('jmarriagefamily', 62, 2000, '147'): "10.1111/j.1741-3737.2000.00147.x",
    ('jmarriagefamily', 62, 2000, '175'): "10.1111/j.1741-3737.2000.00175.x",
    ('jmarriagefamily', 62, 2000, '162'): "10.1111/j.1741-3737.2000.00162.x",

    ('socieduc', 76, 2003, '368'): "", # not an article, didn't know how to proceed
    ('socieduc', 76, 2003, '189'): "", # not an article, didn't know how to proceed
    ('socieduc', 75, 2002, '97'): "", # not an article (in memoriam), didn't know how to proceed
    ('socieduc', 73, 2000, '92'): "10.2307/2673239",
    ('socieduc', 74, 2001, '135'): "10.2307/2673258",
    ('socieduc', 74, 2001, '100'): "10.2307/2673256",
    ('socieduc', 74, 2001, '115'): "10.2307/2673257",
    ('socieduc', 74, 2001, '3'): "10.2307/2673250",
    ('socieduc', 74, 2001, '35'): "10.2307/2673252",
    ('socieduc', 74, 2001, '71'): "10.2307/2673254",
    ('socieduc', 74, 2001, '159'): "10.2307/2673260",
    ('socieduc', 74, 2001, '154'): "10.2307/2673259",
    ('socieduc', 74, 2001, '178'): "10.2307/2673262",
    ('socieduc', 74, 2001, '50'): "10.2307/2673253",
    ('socieduc', 74, 2001, '169'): "10.2307/2673261",
    ('socieduc', 74, 2001, '19'): "10.2307/2673251",
    ('socieduc', 74, 2001, '88'): "10.2307/2673255",

    ('socimeth', 39, 2009, '1'): "10.1111/j.1467-9531.2009.01214.x",
    ('socimeth', 38, 2008, '1'): "10.1111/j.1467-9531.2008.00199.x",

    ('socsciresearch', 38, 2009, '1'): "10.1016/j.ssresearch.2008.03.007",
    ('socsciresearch', 38, 2009, '19'): "10.1016/j.ssresearch.2008.05.003",
    ('socsciresearch', 38, 2009, '29'): "10.1016/j.ssresearch.2008.07.004",
    ('socsciresearch', 38, 2009, '39'): "10.1016/j.ssresearch.2008.09.004",
    ('socsciresearch', 38, 2009, '55'): "10.1016/j.ssresearch.2008.10.002",
    ('socsciresearch', 38, 2009, '71'): "10.1016/j.ssresearch.2008.10.001",
    ('socsciresearch', 38, 2009, '86'): "10.1016/j.ssresearch.2008.08.003",
    ('socsciresearch', 38, 2009, '103'): "10.1016/j.ssresearch.2008.08.005",
    ('socsciresearch', 38, 2009, '118'): "10.1016/j.ssresearch.2008.07.002",
    ('socsciresearch', 38, 2009, '134'): "10.1016/j.ssresearch.2008.09.001",
    ('socsciresearch', 38, 2009, '146'): "10.1016/j.ssresearch.2008.08.001"
}


# Function to manually add DOIs
def add_manual_dois(row):
    key = (row["journal"], row["volume"], row["year"], row["BP"])
    return manual_doi_mapping.get(key, row["DI"])  # Keep original DOI if already present

# Apply manual mapping to the missing DOIs
df_join_updated["DI"] = df_join_updated.apply(add_manual_dois, axis=1)

num_missing_doi_after_fix = df_join_updated["DI"].isna().sum() + len(df_join_updated[df_join_updated["DI"] == ""])
print(f"Total articles still missing DOI after manual fix: {num_missing_doi_after_fix}")

missing_doi_tuples = df_join_updated[df_join_updated["DI"] == ""][["journal", "volume", "year", "BP"]]

for row in missing_doi_tuples.itertuples(index=False, name=None):
    print(row)  # Prints as tuple (journal, volume, year, beginning_page)



Total articles still missing DOI after manual fix: 3
('socieduc', 76, 2003, '368')
('socieduc', 76, 2003, '189')
('socieduc', 75, 2002, '97')


<ipython-input-23-d4c01ad7592e>:277: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_join_updated["DI"] = df_join_updated.apply(add_manual_dois, axis=1)


In [24]:
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.regressionplots import abline_plot
import seaborn as sns
from statsmodels.formula.api import rlm

In [25]:
from Bio import Entrez
import xml.etree.ElementTree as ET

def get_pubmed_id_from_doi(doi):
    Entrez.email = "akshu.enaganti@gmail.com"  # Replace with your email

    try:
        handle = Entrez.esearch(db="pubmed", term=doi, retmax=1)
        record = Entrez.read(handle)
        handle.close()

        if record["Count"] == '0':
            return None

        pubmed_id = record["IdList"][0]
        return pubmed_id

    except Exception as e:
        print(f"Error: {e}")
        return None

In [26]:
import requests
import xml.etree.ElementTree as ET
import time

def get_pmid_from_doi(doi):
    """Retrieves the PMID from a given DOI using the NCBI E-utilities API."""
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        "db": "pubmed",
        "retmode": "xml",
        "term": doi + "[DOI]"
    }
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        xml_content = response.text
        root = ET.fromstring(xml_content)
        
        id_list = root.findall(".//Id")
        if id_list:
             return id_list[0].text
        else:
            return None
    except requests.exceptions.RequestException as e:
        print(f"Request error for DOI {doi}: {e}")
        return None
    except ET.ParseError as e:
         print(f"XML Parse error for DOI {doi}: {e}")
         return None

def get_pmids_from_dois(doi_list):
    """Retrieves PMIDs for a list of DOIs."""
    pmid_dict = {}
    for doi in doi_list:
        pmid = get_pmid_from_doi(doi)
        if pmid:
            pmid_dict[doi] = pmid
        else:
            pmid_dict[doi] = "PMID not found"
        time.sleep(0.333)  # Be mindful of NCBI rate limits (3 requests per second)
    return pmid_dict

In [27]:
dois = ["10.1016/j.cell.2020.01.008", "10.1038/s41586-020-2001-x", "10.1126/science.aba7269"]
results = get_pmids_from_dois(dois)
for doi, pmid in results.items():
    print(f"DOI: {doi}, PMID: {pmid}")

DOI: 10.1016/j.cell.2020.01.008, PMID: 32004463
DOI: 10.1038/s41586-020-2001-x, PMID: 32076266
DOI: 10.1126/science.aba7269, PMID: PMID not found


In [28]:
from pyalex import Works, Authors, Sources, Institutions, Topics, Publishers, Funders
from pyalex import config
import sys
import time
import json

In [29]:

config.email = "akshu.enaganti@gmail.com"

config.max_retries = 0
config.retry_backoff_factor = 0.1
config.retry_http_codes = [429, 500, 503]

doi = "10.1111/j.1741-3737.2008.00479.x"
w = Works().filter(doi=doi).get()

# Print the retrieved work
print(json.dumps(w, indent=2))

[
  {
    "id": "https://openalex.org/W2068446507",
    "doi": "https://doi.org/10.1111/j.1741-3737.2008.00479.x",
    "title": "Life Course Transitions and Housework: Marriage, Parenthood, and Time on Housework",
    "display_name": "Life Course Transitions and Housework: Marriage, Parenthood, and Time on Housework",
    "publication_year": 2008,
    "publication_date": "2008-04-07",
    "ids": {
      "openalex": "https://openalex.org/W2068446507",
      "doi": "https://doi.org/10.1111/j.1741-3737.2008.00479.x",
      "mag": "2068446507"
    },
    "language": "en",
    "primary_location": {
      "is_oa": false,
      "landing_page_url": "https://doi.org/10.1111/j.1741-3737.2008.00479.x",
      "pdf_url": null,
      "source": {
        "id": "https://openalex.org/S146344",
        "display_name": "Journal of Marriage and Family",
        "issn_l": "0022-2445",
        "issn": [
          "0022-2445",
          "1741-3737"
        ],
        "is_oa": false,
        "is_in_doaj": fal

In [30]:
from datetime import datetime
config.email = "akshu.enaganti@gmail.com"
works = Works()

work_ids = sorted(["https://doi.org/"+i for i in list(df4['DI'])])
alex_results = []
alex_failed = []
out_d = "/Users/akshuenaganti/Downloads"

# New: Progress tracking variables
start_time = time.time()
total_works = len(work_ids)
print(f"Started processing {total_works} DOIs at {datetime.now().strftime('%H:%M:%S')}")

for idx, i in enumerate(work_ids):
    try:
        # New: Print every 100 items for real-time feedback
        if idx % 100 == 0:
            elapsed = time.time() - start_time
            print(f"Processing item {idx+1}/{total_works} | Elapsed: {elapsed:.1f}s")
        
        w = works[i]
        openalex_id = w['id'].split('/')[-1]
        alex_results.append([str(idx), i, openalex_id, json.dumps(w)])
        
    except Exception as e:
        error_msg = f"Failed for {i}: {str(e)}"
        print(error_msg)
        alex_failed.append([str(idx), i, "None", error_msg])
    
    time.sleep(0.3)
    
    # Save progress every 100 items (with timestamp)
    if idx % 100 == 99:
        batch_num = idx // 100
        print(f"💾 Saving batch {batch_num} at {datetime.now().strftime('%H:%M:%S')}")
        with open(os.path.join(out_d, f"AlexWorks_{batch_num}.txt"), 'w', encoding='utf-8') as f1:
            f1.write("\n".join(["\t".join(row) for row in alex_results]))
        # ... (same code for saving failed items)

# Final batch save (with completion message)
if alex_results or alex_failed:
    print(f"💾 Saving final batch at {datetime.now().strftime('%H:%M:%S')}")
    # ... (same final save code)

# Completion summary
total_time = time.time() - start_time
print(f"Completed; processed {total_works} DOIs in {total_time/60:.1f} minutes")

Started processing 3355 DOIs at 19:09:43
Processing item 1/3355 | Elapsed: 0.0s
💾 Saving batch 0 at 19:10:26
Processing item 101/3355 | Elapsed: 42.3s
💾 Saving batch 1 at 19:11:08
Processing item 201/3355 | Elapsed: 84.7s
💾 Saving batch 2 at 19:11:52
Processing item 301/3355 | Elapsed: 128.2s
💾 Saving batch 3 at 19:12:45
Processing item 401/3355 | Elapsed: 181.2s
💾 Saving batch 4 at 19:13:55
Processing item 501/3355 | Elapsed: 252.0s
💾 Saving batch 5 at 19:15:06
Processing item 601/3355 | Elapsed: 322.4s
💾 Saving batch 6 at 19:16:10
Processing item 701/3355 | Elapsed: 386.2s
Failed for https://doi.org/10.1086/378035: 404 Client Error: NOT FOUND for url: https://api.openalex.org/works/https://doi.org/10.1086/378035
💾 Saving batch 7 at 19:17:32
Processing item 801/3355 | Elapsed: 469.1s
💾 Saving batch 8 at 19:18:31
Processing item 901/3355 | Elapsed: 527.8s
💾 Saving batch 9 at 19:19:30
Processing item 1001/3355 | Elapsed: 586.5s
💾 Saving batch 10 at 19:20:24
Processing item 1101/3355 | E

In [94]:
out_d = "/Users/akshuenaganti/Downloads"

# Initialize variables
doi_mag = []
n_no_mag = 0

for filename in os.listdir(out_d):
    if filename.startswith("AlexWorks_") and filename.endswith(".txt"):
        with open(os.path.join(out_d, filename), 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split('\t')
                
                # Handle both 3-part and 4-part lines
                if len(parts) == 4:  # New format: [idx, doi_url, openalex_id, json_record]
                    _, doi_url, _, json_record = parts
                elif len(parts) == 3:  # Old format: [idx, doi_url, json_record]
                    _, doi_url, json_record = parts
                else:
                    print(f"Skipping malformed line in {filename}: {line.strip()}")
                    continue
                
                # Extract DOI
                doi = doi_url.split("https://doi.org/")[-1].strip()
                
                # Skip if DOI is missing
                if not doi:
                    continue
                
                # Parse JSON
                try:
                    record = json.loads(json_record)
                    ids = record.get('ids', {})  # Handle missing 'ids' key
                    mag = ids.get('mag')
                    
                    if not mag:
                        n_no_mag += 1
                        mag = None
                    
                    doi_mag.append([doi, mag])
                    
                except json.JSONDecodeError:
                    n_no_mag += 1
                    doi_mag.append([doi, None])

# Create DataFrame and remove duplicates
df_doimag = pd.DataFrame(doi_mag, columns=["DI", "MAGpaperID"]).drop_duplicates()
df_doimag = df_doimag.drop_duplicates(subset=["DI"], keep='first')


print("\nFinal Results:")
print(f"DOIs that have no corresponding MAG IDs: {n_no_mag}")
print(f"Total unique DOI-MAG pairs: {len(df_doimag)}")
print(df_doimag.head())



Final Results:
DOIs that have no corresponding MAG IDs: 510
Total unique DOI-MAG pairs: 3354
                       DI  MAGpaperID
0  10.1006/aphy.2002.6256        None
1  10.1006/ssre.1999.0649  2086452099
2  10.1006/ssre.1999.0651  2008907153
3  10.1006/ssre.1999.0652  2023663235
4  10.1006/ssre.1999.0654  1978683896


In [95]:
df5 = df4.merge(df_doimag, on="DI", how = "left")
df5

,artid,DI,journal,volume,issue,year,month,pages,cited3,cited2012,...,aut_flowhier,ack_flowhier,com_flowhier,aut_numcomp,ack_numcomp,com_numcomp,bip_numcomp,BP,EP,MAGpaperID
0,1,10.1146/annurev-soc-070308-115936,annurevisoci,35,1,2009,6,1-19,0,0,...,99.0,99.0,99.000000,1,0,1,1,1,19,2117643286
1,2,10.1146/annurev-soc-070308-115916,annurevisoci,35,1,2009,6,21-42,15,15,...,99.0,99.0,99.000000,1,0,1,1,21,42,2137925630
2,3,10.1146/annurev-soc-070308-115954,annurevisoci,35,1,2009,6,43-65,17,21,...,1.0,0.2,0.277778,1,3,3,5,43,65,2138260574
3,4,10.1146/annurev.soc.34.040507.134631,annurevisoci,35,1,2009,6,67-85,6,7,...,99.0,99.0,0.000000,1,2,2,2,67,85,2131047368
4,5,10.1146/annurev-soc-070308-115920,annurevisoci,35,1,2009,6,87-105,18,18,...,0.0,99.0,0.000000,1,0,1,2,87,105,2103375828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3350,3746,10.2307/2657293,amersocirevi,65,1,2000,2,122-127,3,21,...,0.0,99.0,0.000000,1,1,1,1,122,127,2318225990
3351,3747,10.2307/2657295,amersocirevi,65,1,2000,2,138-167,8,54,...,99.0,99.0,0.000000,1,5,5,5,138,167,2325384519
3352,3748,10.2307/2657294,amersocirevi,65,1,2000,2,128-137,1,7,...,1.0,99.0,1.000000,3,1,4,5,128,137,2324925852
3353,3749,10.2307/2657291,amersocirevi,65,1,2000,2,96-116,16,186,...,0.0,99.0,0.000000,1,1,1,1,96,116,2003629458


In [97]:
disruption_file = "dataverse_files/Aggregated_20210521.txt"

disruption_df = pd.read_csv(disruption_file, sep='\t', names=['MAGpaperID','year','field','teamsize','citation','disruption'],dtype={"MAGpaperID":str})

# Display the first few rows to verify the data
print("First few rows of the disruption dataset:")
display(disruption_df.head())

print("\nDataset info:")
display(disruption_df.info())


First few rows of the disruption dataset:


,MAGpaperID,year,field,teamsize,citation,disruption
0,1970392578,1830,10,1,3,0.750000
1,2108276706,1842,5,1,1,0.333333
2,2022566795,1846,7,1,12,0.200000
3,2065789632,1850,9,1,3,0.150000
4,219463075,1851,5,1,3,0.214286



Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19495814 entries, 0 to 19495813
Data columns (total 6 columns):
 #   Column      Dtype  
---  ------      -----  
 0   MAGpaperID  object 
 1   year        int64  
 2   field       int64  
 3   teamsize    int64  
 4   citation    int64  
 5   disruption  float64
dtypes: float64(1), int64(4), object(1)
memory usage: 892.4+ MB


None

In [ ]:
df6 = df5.merge(disruption_df, on="MAGpaperID", how="left")
df6

"""
Recheck missing DOI's from failed fetches via OpenAlex
Track all changes in length of dataframes over course of changes + include explanations
Fetch missing authors from openalex and figure out formatting to match from WoS datasets
"""


,artid,DI,journal,volume,issue,year_x,month,pages,cited3,cited2012,...,com_numcomp,bip_numcomp,BP,EP,MAGpaperID,year_y,field,teamsize,citation,disruption
0,1,10.1146/annurev-soc-070308-115936,annurevisoci,35,1,2009,6,1-19,0,0,...,1,1,1,19,2117643286,2009.0,17.0,1.0,8.0,0.001887
1,2,10.1146/annurev-soc-070308-115916,annurevisoci,35,1,2009,6,21-42,15,15,...,1,1,21,42,2137925630,2009.0,17.0,1.0,174.0,-0.000445
2,3,10.1146/annurev-soc-070308-115954,annurevisoci,35,1,2009,6,43-65,17,21,...,3,5,43,65,2138260574,2009.0,17.0,2.0,181.0,-0.002326
3,4,10.1146/annurev.soc.34.040507.134631,annurevisoci,35,1,2009,6,67-85,6,7,...,2,2,67,85,2131047368,2009.0,17.0,1.0,25.0,0.000206
4,5,10.1146/annurev-soc-070308-115920,annurevisoci,35,1,2009,6,87-105,18,18,...,1,2,87,105,2103375828,2009.0,17.0,2.0,308.0,-0.014234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3350,3746,10.2307/2657293,amersocirevi,65,1,2000,2,122-127,3,21,...,1,1,122,127,2318225990,NaN,NaN,NaN,NaN,NaN
3351,3747,10.2307/2657295,amersocirevi,65,1,2000,2,138-167,8,54,...,5,5,138,167,2325384519,NaN,NaN,NaN,NaN,NaN
3352,3748,10.2307/2657294,amersocirevi,65,1,2000,2,128-137,1,7,...,4,5,128,137,2324925852,NaN,NaN,NaN,NaN,NaN
3353,3749,10.2307/2657291,amersocirevi,65,1,2000,2,96-116,16,186,...,1,1,96,116,2003629458,NaN,NaN,NaN,NaN,NaN


In [100]:
print(df6.columns)
df6[['artid', 'DI', 'cited3', 'cited2012', 'aut_num', 'ack_num', 'com_num', 'teamsize', 'citation', 'disruption']].head(50)

Index(['artid', 'DI', 'journal', 'volume', 'issue', 'year_x', 'month', 'pages',
       'cited3', 'cited2012', 'aut_num', 'ack_num', 'com_num', 'aut_cluster',
       'ack_cluster', 'com_cluster', 'bip_cluster', 'aut_jaccard',
       'ack_jaccard', 'com_jaccard', 'aut_density', 'ack_density',
       'com_density', 'bip_density', 'aut_avgdegree', 'ack_avgdegree',
       'com_avgdegree', 'bip_avgdegree', 'aut_flowhier', 'ack_flowhier',
       'com_flowhier', 'aut_numcomp', 'ack_numcomp', 'com_numcomp',
       'bip_numcomp', 'BP', 'EP', 'MAGpaperID', 'year_y', 'field', 'teamsize',
       'citation', 'disruption'],
      dtype='object')


,artid,DI,cited3,cited2012,aut_num,ack_num,com_num,teamsize,citation,disruption
0,1,10.1146/annurev-soc-070308-115936,0,0,1,0,1,1.0,8.0,0.001887
1,2,10.1146/annurev-soc-070308-115916,15,15,1,0,1,1.0,174.0,-0.000445
2,3,10.1146/annurev-soc-070308-115954,17,21,2,7,9,2.0,181.0,-0.002326
3,4,10.1146/annurev.soc.34.040507.134631,6,7,1,2,3,1.0,25.0,0.000206
4,5,10.1146/annurev-soc-070308-115920,18,18,2,0,2,2.0,308.0,-0.014234
5,6,10.1146/annurev-soc-070308-120040,11,13,2,1,3,2.0,57.0,-0.001548
6,7,10.1146/annurev.soc.34.040507.134657,7,7,2,0,2,2.0,59.0,-0.004239
7,8,10.1146/annurev-soc-070308-115944,10,11,1,0,1,1.0,195.0,-0.007928
8,9,10.1146/annurev-soc-070308-120001,5,5,2,2,4,2.0,30.0,-0.002033
9,10,10.1146/annurev.soc.34.040507.134615,20,23,1,0,1,1.0,161.0,-0.011953


In [101]:
form_list00 = ["disruption", "ack_num", 'aut_num', 'citation'] #'cited3', 'cited2012'
#form_list00 = ["disruption", "ack_num"]
X1=df6[form_list00]
X1=X1.dropna()
X1
#print(len(X1))
form = 'disruption ~' + " + ".join(form_list00[1:])
model1=smf.ols(formula = form, data=X1).fit()
predictions = model1.predict(X1)
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             disruption   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     10.62
Date:                Wed, 19 Feb 2025   Prob (F-statistic):           6.18e-07
Time:                        11:57:53   Log-Likelihood:                 6239.2
No. Observations:                2532   AIC:                        -1.247e+04
Df Residuals:                    2528   BIC:                        -1.245e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0050      0.001      4.944      0.000       0.003       0.007
ack_num       -0.0007      0.000     -4.177      0.000      -0.001      -0.000
aut_num       -0.0020      0.000     -4.534      0.000      -0.003      -0.001
citation    2.218e-06   2.94e-06      0.755      0.451   -3.55e-06    7.98e-06
==============================================================================
Omnibus:                     4530.507   Durbin-Watson:                   1.763
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          5040429.855
Skew:                          12.708   Prob(JB):                         0.00
Kurtosis:                     220.096   Cond. No.                         428.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [102]:
X1=df6[form_list00]
X1=X1.dropna()
X1
#print(len(X1))
form = 'disruption ~' + " + ".join(form_list00[1:])
form += " + ack_num * aut_num"
model1=smf.ols(formula = form, data=X1).fit()
predictions = model1.predict(X1)
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             disruption   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     8.482
Date:                Wed, 19 Feb 2025   Prob (F-statistic):           8.52e-07
Time:                        11:57:56   Log-Likelihood:                 6240.2
No. Observations:                2532   AIC:                        -1.247e+04
Df Residuals:                    2527   BIC:                        -1.244e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.0057      0.001      5.086      0.000       0.003       0.008
ack_num            -0.0012      0.000     -3.244      0.001      -0.002      -0.000
aut_num            -0.0023      0.001     -4.618      0.000      -0.003      -0.001
citation         2.308e-06   2.94e-06      0.785      0.432   -3.46e-06    8.07e-06
ack_num:aut_num     0.0003      0.000      1.437      0.151      -0.000       0.001
==============================================================================
Omnibus:                     4528.285   Durbin-Watson:                   1.766
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          5025165.883
Skew:                          12.695   Prob(JB):                         0.00
Kurtosis:                     219.765   Cond. No.                         487.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [103]:
print(df6.columns)
print(df_join_updated.columns)

Index(['artid', 'DI', 'journal', 'volume', 'issue', 'year_x', 'month', 'pages',
       'cited3', 'cited2012', 'aut_num', 'ack_num', 'com_num', 'aut_cluster',
       'ack_cluster', 'com_cluster', 'bip_cluster', 'aut_jaccard',
       'ack_jaccard', 'com_jaccard', 'aut_density', 'ack_density',
       'com_density', 'bip_density', 'aut_avgdegree', 'ack_avgdegree',
       'com_avgdegree', 'bip_avgdegree', 'aut_flowhier', 'ack_flowhier',
       'com_flowhier', 'aut_numcomp', 'ack_numcomp', 'com_numcomp',
       'bip_numcomp', 'BP', 'EP', 'MAGpaperID', 'year_y', 'field', 'teamsize',
       'citation', 'disruption'],
      dtype='object')
Index(['artid', 'journal', 'volume', 'issue', 'year', 'month', 'pages_x',
       'cited3', 'cited2012', 'aut_num', 'ack_num', 'com_num', 'aut_cluster',
       'ack_cluster', 'com_cluster', 'bip_cluster', 'aut_jaccard',
       'ack_jaccard', 'com_jaccard', 'aut_density', 'ack_density',
       'com_density', 'bip_density', 'aut_avgdegree', 'ack_avgdegree',
    

In [130]:
df6 = df6.drop_duplicates(subset=['DI'])
df_join_updated = df_join_updated.drop_duplicates(subset=['DI'])

# Then merge the deduplicated DataFrames
merged_df = pd.merge(
    df6, 
    df_join_updated, 
    left_on=['DI'], 
    right_on=['DI'], 
    how='inner'
)

# Remove any remaining duplicates if they exist
merged_df = merged_df.drop_duplicates(subset=['DI'])

# Verify the result
#print(merged_df.columns)
merged_df

,artid_x,DI,journal_x,volume_x,issue_x,year_x,month_x,pages,cited3_x,cited2012_x,...,D2,AU,TI,SO,PY,VL,IS,EP_y,pages_y,pages_diff
0,1,10.1146/annurev-soc-070308-115936,annurevisoci,35,1,2009,6,1-19,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-19,False
1,2,10.1146/annurev-soc-070308-115916,annurevisoci,35,1,2009,6,21-42,15,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21-42,False
2,3,10.1146/annurev-soc-070308-115954,annurevisoci,35,1,2009,6,43-65,17,21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43-65,False
3,4,10.1146/annurev.soc.34.040507.134631,annurevisoci,35,1,2009,6,67-85,6,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67-85,False
4,5,10.1146/annurev-soc-070308-115920,annurevisoci,35,1,2009,6,87-105,18,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87-105,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3349,3746,10.2307/2657293,amersocirevi,65,1,2000,2,122-127,3,21,...,NaN,"Frank, DJ; Hironaka, A; Schofer, E",Environmentalism as a global institution -: <i...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,127,122-127,False
3350,3747,10.2307/2657295,amersocirevi,65,1,2000,2,138-167,8,54,...,NaN,"Gorski, PS",Historicizing the secularization debate: Churc...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,167,138-167,False
3351,3748,10.2307/2657294,amersocirevi,65,1,2000,2,128-137,1,7,...,NaN,"Horton, HD; Allen, BL; Herring, C; Thomas, ME",Lost in the storm: The sociology of the black ...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,137,128-137,False
3352,3749,10.2307/2657291,amersocirevi,65,1,2000,2,96-116,16,186,...,NaN,"Frank, DJ; Hironaka, A; Schofer, E",The nation-state and the natural environment o...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,116,96-116,False


In [ ]:
# Example: If authors are stored as a list or semicolon-separated string
merged_df['AU'] = df_join_updated['AU'].str.split(';')  # Adjust based on your data format
merged_df = merged_df.explode('AU').reset_index(drop=True)

# Check the exploded DataFrame
#print(merged_df[['journal_x', 'issue_x', 'year_x', 'AU', 'disruption']].head())



,artid_x,DI,journal_x,volume_x,issue_x,year_x,month_x,pages,cited3_x,cited2012_x,...,D2,AU,TI,SO,PY,VL,IS,EP_y,pages_y,pages_diff
0,1,10.1146/annurev-soc-070308-115936,annurevisoci,35,1,2009,6,1-19,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-19,False
1,2,10.1146/annurev-soc-070308-115916,annurevisoci,35,1,2009,6,21-42,15,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21-42,False
2,3,10.1146/annurev-soc-070308-115954,annurevisoci,35,1,2009,6,43-65,17,21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43-65,False
3,4,10.1146/annurev.soc.34.040507.134631,annurevisoci,35,1,2009,6,67-85,6,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67-85,False
4,5,10.1146/annurev-soc-070308-115920,annurevisoci,35,1,2009,6,87-105,18,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87-105,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9119,3748,10.2307/2657294,amersocirevi,65,1,2000,2,128-137,1,7,...,NaN,NaN,Lost in the storm: The sociology of the black ...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,137,128-137,False
9120,3749,10.2307/2657291,amersocirevi,65,1,2000,2,96-116,16,186,...,NaN,NaN,The nation-state and the natural environment o...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,116,96-116,False
9121,3749,10.2307/2657291,amersocirevi,65,1,2000,2,96-116,16,186,...,NaN,NaN,The nation-state and the natural environment o...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,116,96-116,False
9122,3750,10.2307/2657290,amersocirevi,65,1,2000,2,77-95,8,80,...,NaN,NaN,Trade globalization since 1795: Waves of integ...,AMERICAN SOCIOLOGICAL REVIEW,2000.0,65.0,1.0,95,77-95,False


In [106]:
# Group by author and compute summary stats
author_disruption = merged_df.groupby('AU')['disruption'].agg(
    mean_disruption='mean',
    std_disruption='std',
    min_disruption='min',
    max_disruption='max',
    median_disruption='median'
).reset_index()

# Check the author disruption summary
print(author_disruption.head())

                   AU  mean_disruption  std_disruption  min_disruption  \
0          Aassve, A.         0.000662             NaN        0.000662   
1            Abbey, A         0.000561             NaN        0.000561   
2   Aber, J. Lawrence         0.001014             NaN        0.001014   
3     Acker, Caroline        -0.000136             NaN       -0.000136   
4         Ackerman, J        -0.001540             NaN       -0.001540   

   max_disruption  median_disruption  
0        0.000662           0.000662  
1        0.000561           0.000561  
2        0.001014           0.001014  
3       -0.000136          -0.000136  
4       -0.001540          -0.001540  


In [107]:
# Calculate z-scores
mean_overall = author_disruption['mean_disruption'].mean()
std_overall = author_disruption['mean_disruption'].std()
author_disruption['z_score'] = (author_disruption['mean_disruption'] - mean_overall) / std_overall

# Check z-scores
print(author_disruption.head())

                   AU  mean_disruption  std_disruption  min_disruption  \
0          Aassve, A.         0.000662             NaN        0.000662   
1            Abbey, A         0.000561             NaN        0.000561   
2   Aber, J. Lawrence         0.001014             NaN        0.001014   
3     Acker, Caroline        -0.000136             NaN       -0.000136   
4         Ackerman, J        -0.001540             NaN       -0.001540   

   max_disruption  median_disruption   z_score  
0        0.000662           0.000662  0.050374  
1        0.000561           0.000561  0.044588  
2        0.001014           0.001014  0.070777  
3       -0.000136          -0.000136  0.004249  
4       -0.001540          -0.001540 -0.076982  


In [108]:
# Filter authors with z > 1
high_disruption_authors = author_disruption[author_disruption['z_score'] > 1]
print("Authors with z > 1:\n", high_disruption_authors)

Authors with z > 1:
                         AU  mean_disruption  std_disruption  min_disruption  \
75               Baker, WE         0.017200             NaN        0.017200   
128    Berger, Lawrence M.         0.250000             NaN        0.250000   
132            Bernard, HR         0.029222        0.044383       -0.002162   
204               Brody, G         0.023729             NaN        0.023729   
222            Brückner, H         0.094595             NaN        0.094595   
...                    ...              ...             ...             ...   
4240     Stamps, Katherine         0.021567             NaN        0.021567   
4287               Sun, YM         0.027146        0.047456       -0.000674   
4317              Thye, SR         0.136752             NaN        0.136752   
4351              Uggen, C         0.021610             NaN        0.021610   
4458         Whitmeyer, JM         0.022813             NaN        0.022813   

      max_disruption  median_d

In [109]:
print("Z-scores summary:\n", author_disruption['z_score'].describe())


Z-scores summary:
 count    3623.000000
mean        0.000000
std         1.000000
min        -5.766128
25%        -0.149594
50%        -0.034961
75%         0.015099
max        25.320294
Name: z_score, dtype: float64


In [127]:
# Try a lower threshold (e.g., 0.5)
high_disruption_authors = author_disruption[(author_disruption['z_score'] > 0.5) & (author_disruption['std_disruption'].notna())].sort_values(
    by='mean_disruption', 
    ascending=False
)

print(f"Number of authors w/disruption z-score > 0.5 and 2+ articles: {len(high_disruption_authors)}")
high_disruption_authors

Number of authors w/disruption z-score > 0.5 and 2+ articles: 37


,AU,mean_disruption,std_disruption,min_disruption,max_disruption,median_disruption,z_score
3620,"Luster, T",0.217840,0.310646,-0.001820,0.437500,0.217840,12.613555
744,"Greenstein, TN",0.129417,0.183624,-0.000424,0.259259,0.129417,7.498556
2403,"Atkinson, MP",0.128757,0.184559,-0.001746,0.259259,0.128757,7.460330
2883,"Edwards, ME",0.098537,0.137341,0.001422,0.195652,0.098537,5.712213
450,"DeBoer, DD",0.093897,0.133185,-0.000279,0.188073,0.093897,5.443798
2764,"Crouter, AC",0.084773,0.120626,-0.000522,0.170068,0.084773,4.915989
824,"Head, MR",0.084773,0.120626,-0.000522,0.170068,0.084773,4.915989
2646,"Carlson, MJ",0.077336,0.032572,0.054304,0.100368,0.077336,4.485791
3920,"Pettit, B",0.062042,0.077230,0.007432,0.116652,0.062042,3.601060
4174,"Simons, RL",0.061482,0.096079,-0.006456,0.129421,0.061482,3.568689
